In [1]:
import json
from pathlib import Path
import numpy as np

result_file = Path(f"button_usage.jsonl")
if result_file.exists():
    data = list(map(lambda x: json.loads(x), open(result_file).readlines()))

    print(f"The number of collections: {len(data)}")

    learn_costs = np.array([x["learn_costs"] for x in data])
    review_costs = np.array([x["review_costs"] for x in data])

    learn_costs_mean = np.median(learn_costs, axis=0).round(3).tolist()
    review_costs_mean = np.median(review_costs, axis=0).round(3).tolist()
    print(f"Learn costs median: {learn_costs_mean}")
    print(f"Review costs median: {review_costs_mean}")

    first_rating_probs = np.array([x["first_rating_prob"] for x in data])
    review_rating_probs = np.array([x["review_rating_prob"] for x in data])

    first_rating_probs_mean = np.mean(first_rating_probs, axis=0).round(3).tolist()
    review_rating_probs_mean = np.nanmean(review_rating_probs, axis=0).round(3).tolist()
    print(f"First rating prob mean: {first_rating_probs_mean}")
    print(f"Review rating prob mean: {review_rating_probs_mean}")

    first_rating_offsets = np.array([x["first_rating_offset"] for x in data])
    first_session_lens = np.array([x["first_session_len"] for x in data])
    
    first_rating_offsets_median = np.median(first_rating_offsets, axis=0).round(3).tolist()
    first_session_lens_median = np.median(first_session_lens, axis=0).round(3).tolist()
    print(f"First rating offset median: {first_rating_offsets_median}")
    print(f"First session len median: {first_session_lens_median}")

    forget_rating_offsets = np.array([x["forget_rating_offset"] for x in data])
    forget_session_lens = np.array([x["forget_session_len"] for x in data])

    forget_rating_offsets_median = np.median(forget_rating_offsets).round(3)
    forget_session_lens_median = np.median(forget_session_lens).round(3)
    print(f"Forget rating offset median: {forget_rating_offsets_median}")
    print(f"Forget session len median: {forget_session_lens_median}")

The number of collections: 19998
Learn costs median: [33.79, 24.3, 13.68, 6.5]
Review costs median: [23.0, 11.68, 7.33, 5.6]
First rating prob mean: [0.24, 0.094, 0.495, 0.171]
Review rating prob mean: [0.224, 0.631, 0.145]
First rating offset median: [-0.72, -0.15, -0.01, 0.0]
First session len median: [2.02, 1.28, 0.81, 0.0]
Forget rating offset median: -0.28
Forget session len median: 1.05
